# Solve a Nonlinear 1D Reaction-Diffusion Equation with Random Coefficients
This notebook solves the PDE:

$$
\frac{\partial u}{\partial t} - D \frac{\partial^2 u}{\partial x^2} + g(x) u^3 = f(x), \quad t \in [0, 1],\ x \in [-1, 1]
$$

with:

### Initial condition:
$$
u(0, x) = 0.5 \cos^2(\pi x)
$$

### Boundary conditions:
$$
u(t, -1) = u(t, 1) = 0.5
$$

### Reaction coefficient:
$$
g(x) = 0.2 + e^{r_1 x} \cos^2(r_2 x), \quad r_1 \sim \mathcal{U}(0.5, 1), \quad r_2 \sim \mathcal{U}(3, 4)
$$

### Forcing term:
$$
f(x) = \exp\left( -\frac{(x - 0.25)^2}{2 k_1^2} \right) \sin^2(k_2 x), \quad k_1 \sim \mathcal{U}(0.2, 0.8), \quad k_2 \sim \mathcal{U}(1, 4)
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set random seed for reproducibility
rng = np.random.default_rng(seed=42)

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

def numerical_solution(
    t: np.ndarray,
    x: np.ndarray,
    p: np.ndarray,
    D: float = 0.01
) -> np.ndarray:
    
    dx = x[1] - x[0]
    Nx = len(x)
    t_eval = t

    r1, r2, k1, k2 = p

    g = 0.2 + np.exp(r1 * x) * np.cos(r2 * x)**2
    f = np.exp(-((x - 0.25)**2) / (2 * k1**2)) * np.sin(k2 * x)**2

    u0 = 0.5 * np.cos(np.pi * x)**2

    def apply_bc(u):
        u[0] = 0.5
        u[-1] = 0.5
        return u

    def laplacian_matrix(N, dx):
        L = np.zeros((N, N))
        for i in range(1, N - 1):
            L[i, i - 1] = 1
            L[i, i] = -2
            L[i, i + 1] = 1
        return L / dx**2

    L_mat = laplacian_matrix(Nx, dx)

    def rhs(ti, u):
        u = apply_bc(u.copy())
        du = D * (L_mat @ u) #- g * u**3 + f
        du[0] = 0.0
        du[-1] = 0.0
        return du

    sol = solve_ivp(rhs, [t[0], t[-1]], u0, t_eval=t_eval, method='RK45')

    return sol.y  # shape: (Nx, Nt)


In [ ]:
D = 0.1
T = 15.0

x = np.linspace(-1, 1, 100)
t = np.linspace(0, T, 100)

# Monte Carlo simulation
num_samples = 1
all_solutions = []

for _ in range(num_samples):
    r1 = rng.uniform(0.5, 1.0)
    r2 = rng.uniform(3.0, 4.0)
    k1 = rng.uniform(0.2, 0.8)
    k2 = rng.uniform(1.0, 4.0)

    all_solutions.append(
        numerical_solution(t, x, [r1, r2, k1, k2], D)
    )

U = np.stack(all_solutions, axis=0)  # shape: (samples, space, time)
mean_u = U.mean(axis=0)
std_u = U.std(axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
line, = ax.plot(x, mean_u[:, 0], color='blue', label='Mean')
band = ax.fill_between(
    x,
    mean_u[:, 0] - std_u[:, 0],
    mean_u[:, 0] + std_u[:, 0],
    color='blue',
    alpha=0.3,
)
ax.set_ylim(0, 1.3)
ax.set_xlabel("x")
ax.set_ylabel("u(x,t)")
title = ax.set_title("t = 0.00")
ax.legend()

# Store reference to the current fill_between object
std_patch = [band]

def update(frame):
    global std_patch
    line.set_ydata(mean_u[:, frame])
    std_patch[0].remove()
    std_patch[0] = ax.fill_between(
        x,
        mean_u[:, frame] - std_u[:, frame],
        mean_u[:, frame] + std_u[:, frame],
        color='blue', alpha=0.2
    )
    title.set_text(f"t = {t[frame]:.2f}")
    return line, std_patch[0]

ani = FuncAnimation(fig, update, frames=len(t), interval=60, blit=False)
plt.close()
HTML(ani.to_jshtml())
